In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install scikit-network==0.24.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 49.8 MB/s eta 0:00:00


In [ ]:
# Download the data
!rm *.tsv.gz
!wget -q https://datasets.imdbws.com/name.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.principals.tsv.gz
!wget -q https://datasets.imdbws.com/title.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.akas.tsv.gz
!ls -la

rm: cannot remove '*.tsv.gz': No such file or directory
total 1117372
drwxr-xr-x 1 root root      4096 Mar 23 14:30 .
drwxr-xr-x 1 root root      4096 Mar 23 14:27 ..
drwxr-xr-x 4 root root      4096 Mar 21 22:53 .config
-rw-r--r-- 1 root root 242796056 Mar 22 13:16 name.basics.tsv.gz
drwxr-xr-x 1 root root      4096 Mar 21 22:54 sample_data
-rw-r--r-- 1 root root 299886705 Mar 22 13:16 title.akas.tsv.gz
-rw-r--r-- 1 root root 169705353 Mar 23 13:15 title.basics.tsv.gz
-rw-r--r-- 1 root root 431770513 Mar 22 13:16 title.principals.tsv.gz


In [ ]:
# Load the titles
title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=False).set_index('tconst')[['titleType', 'primaryTitle', 'startYear']]
title.head()

,titleType,primaryTitle,startYear
tconst,,,
tt0000001,short,Carmencita,1894
tt0000002,short,Le clown et ses chiens,1892
tt0000003,short,Pauvre Pierrot,1892
tt0000004,short,Un bon bock,1892
tt0000005,short,Blacksmith Scene,1893


In [ ]:
title = title[title['startYear'].str.find('\\N')==-1]

In [ ]:
title['startYear'] = title.startYear.values.astype('int')

In [ ]:
title = title[title['startYear']>=2000]

In [ ]:
# Load the cast of each film
cast = pd.read_csv('title.principals.tsv.gz', sep='\t')[['tconst', 'nconst', 'category']]
# Only consider actors, not directors, composers, etc. Shrinks data to about 40%
cast = cast[cast.category.isin({'actor', 'actress'})]
cast.head()

In [ ]:
# Only consider movies, not TV series, etc. Shrinks data to ~5%
movies = title[title['titleType'] == 'movie']
cast = cast[cast['tconst'].isin(movies.index)]
# This is what the network looks like
cast.head()

In [ ]:
# Restrict data to just a single region (e.g. IN, US, etc)
# This loads the region for each title
region = pd.read_csv('title.akas.tsv.gz', sep='\t', low_memory=True).set_index('titleId')['region']
region.head(10)
# new_region = region[region['language'] == 'hi']

In [ ]:
# Load the name data along with birth year
name = pd.read_csv('name.basics.tsv.gz', sep='\t', na_values='\\N', dtype={'birthYear': float}).set_index('nconst')[['primaryName', 'birthYear']]

NameError: ignored

In [ ]:
name.head()

NameError: ignored

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
 def get_pairs(lang='IN', min_acted=1, min_pairings=1):
     '''
#     Returns an adjacency matrix and actor mapping of actor pairs where:
#     - Each actor has acted in at least min_acted films
#     - The two actors have acted together in at least min_pairings films
#     - And (optionally), belong to a region `lang` (IN, UN, etc)
#     '''
     graph = cast
     if lang is 'IN':
        graph = graph[graph['tconst'].isin(region[region == lang].index)]
        pairs = pairs.drop(columns=['row', 'col'])
        pairs.columns = ['count', 'name1', 'year1', 'name2', 'year2']
     return pairs.sort_values('count', ascending=False)

     pairs, cat = get_pairs('IN', 1, 1)


In [ ]:
cat

In [ ]:
pairs

In [ ]:
ForKumu = lookup(pairs, cat)
ForKumu

In [ ]:
ForKumu = ForKumu[['name1', 'name2', 'count']]
ForKumu = ForKumu.rename(columns={'name1':'From',
                                  'name2':'To',
                                  'count':'Strength'})
ForKumu

In [ ]:
ForKumu.to_excel("pairs.xlsx", index = False)